# Fine-Tuned

This notebook extracts respiration signals for each subject using the fine-tuned models.

In [ ]:
import os
import numpy as np

models_dirs = [
    os.path.join('..', 'models', 'fine_tuned', '20240504_001813'),
    os.path.join('..', 'models', 'fine_tuned', '20240504_084248'),
    os.path.join('..', 'models', 'fine_tuned', '20240504_132901'),
    os.path.join('..', 'models', 'fine_tuned', '20240504_163423'),
    os.path.join('..', 'models', 'fine_tuned', '20240504_221524'),
    os.path.join('..', 'models', 'fine_tuned', '20240505_094817'),
]

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()
dim = 72
frame_depth = 20

In [ ]:
model_info = []
manifests = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)
    manifests.append(manifest)

    for tuned_model in manifest['tuned_models']:
        model_info.append({
            'id': manifest['id'],
            'base_model': manifest['base_model'],
            'loss_fn': manifest['loss_fn'],
            'model': tuned_model['model'],
            'epoch': tuned_model['epoch'],
            'validation_loss': tuned_model['validation_loss'],
        })

In [ ]:
import pandas as pd

model_df = pd.DataFrame(model_info)
model_df

In [ ]:
import torch
from respiration.extractor.efficient_phys import EfficientPhys


def load_model(path) -> EfficientPhys:
    # Wrap modul in nn.DataParallel
    model = EfficientPhys(img_size=dim, frame_depth=frame_depth)
    # Fix model loading: Some key have an extra 'module.' prefix
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.load_state_dict(torch.load(path, map_location=device))

    _ = model.eval()

    return model

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
def predict_signal(model: EfficientPhys, subject: str, setting: str) -> tuple[np.ndarray, int, int]:
    video_path = dataset.get_video_path(subject, setting)

    # frame_count = utils.get_frame_count(video_path)
    # chunk_size = (frame_count // frame_depth) * frame_depth - (frame_depth - 1)
    chunk_size = frame_depth * 100 + 1

    frames, meta = utils.read_video_rgb(video_path, chunk_size)
    frames = utils.down_sample_video(frames, dim)

    frames = torch.tensor(frames, dtype=torch.float32, device=device).permute(0, 3, 1, 2)

    with torch.no_grad():
        prediction = model(frames).cpu().detach().numpy().squeeze()

    del frames

    return prediction, chunk_size, meta.fps

In [ ]:
import datetime as dt

predictions = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    best_model = manifest['tuned_models'][-1]
    model_path = os.path.join(model_dir, best_model['model'])

    model = load_model(model_path)

    scenarios = []
    scenarios.extend(manifest['training_scenarios'])
    scenarios.extend(manifest['testing_scenarios'])

    for scenario in scenarios:
        if not dataset.contains(scenario[0], scenario[1]):
            print(f'Skipping {scenario[0]} - {scenario[1]}')
            continue

        print(f'Processing {manifest['id']}: {scenario[0]} - {scenario[1]}')

        start = dt.datetime.now()
        prediction, chunks, fps = predict_signal(model, scenario[0], scenario[1])

        predictions.append({
            'id': manifest['id'],
            'subject': scenario[0],
            'setting': scenario[1],
            'duration': dt.datetime.now() - start,
            'chunk_size': chunks,
            'sample_rate': fps,
            'signal': prediction.tolist(),
        })

In [ ]:
prediction_df = pd.DataFrame(predictions)

evaluation_dir = os.path.join('..', 'evaluation', 'fine_tuned')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

evaluation_path = os.path.join(evaluation_dir, 'predictions.csv')
prediction_df.to_csv(evaluation_path, index=False)

manifests_path = os.path.join(evaluation_dir, 'manifests.json')
utils.write_json(manifests_path, manifests)

prediction_df